In [3]:
import pandas as pd
import numpy as np

### groupby

In [ ]:
df1 = pd.DataFrame({'x':range(1,5),'y':range(11,15),'z':list('aabb')}, index=range(4))
"""聚合可以用内部列，也可以用外部等长的列"""
"""部分内置运算函数count,size,sum,mean,median,std,var,min,max,prod,first,last"""
"""df1['x'].groupby(df1['z'])不可简写为df1['x'].groupby(['z'])，只有df1.groupby(['z'])可对df1['z']简写为['z']"""
grouper1 = df1['x'].groupby([df1['z'], list('mnmn')])  # return series
grouper2 = df1[['x']].groupby([df1['z'], list('mnmn')])  # return dataframe
grouper1.count()  # return series
grouper2.count()  # return dataframe

In [5]:
"""语法糖，返回的grouper是iterator，每组数据是1个元祖，元祖里面有2个元素，1是序号(元祖或值)，2是df数据"""
grouper = df1.groupby(['z', list('mnmn')], axis=0)  # 默认axis=0纵向分组, axis=1是横向分组。
dic_grp = dict(list(grouper))  # 字典中每个value都是分组后的df
dic_grp[('a','m')]
list(grouper)[0]
list(grouper)[0][0]
list(grouper)[0][1]
list(grouper)

[(('a', 'm'),    x   y  z
  0  1  11  a), (('a', 'n'),    x   y  z
  1  2  12  a), (('b', 'm'),    x   y  z
  2  3  13  b), (('b', 'n'),    x   y  z
  3  4  14  b)]

In [16]:
"""默认聚合所有列(符合计算条件的)，比如sum计算所有数值列，count计算所有数值列和字符串列"""
grouper.count()

x  y
z        
a m  1  1
  n  1  1
b m  1  1
  n  1  1

In [17]:
"""同df1['x'].groupby([df1['z'], list('mnmn')]).count()"""
grouper['x'].count()

z   
a  m    1
   n    1
b  m    1
   n    1
Name: x, dtype: int64

In [18]:
"""同df1[['x']].groupby([df1['z'], list('mnmn')]).count()"""
grouper[['x']].count()

x
z     
a m  1
  n  1
b m  1
  n  1

In [19]:
"""size统计行数，永远返回一列(所以必返回series)。count将NaN,None,np.nan等计为0(空字符串等未测试)，但size会对整行的NaN,None计为1"""
grouper.size()  # 等价写法grouper[['x','y']].size()， 其实加[['x','y']]多余
"""分组describe"""
grouper.describe()
"""分组quantile"""
grouper.quantile(0.5)

0.5    x     y
z             
a m  1.0  11.0
  n  2.0  12.0
b m  3.0  13.0
  n  4.0  14.0

In [20]:
"""小技巧，切分不同类型的列，对行groupby，用dtypes"""
grp = df1.groupby(df1.dtypes, axis=1)
list(grp)[0][1]

,x,y
0,1,11
1,2,12
2,3,13
3,4,14


In [21]:
list(grp)[1][1]

,z
0,a
1,a
2,b
3,b


### grouper之aggregate只接受聚合函数，也就是返回标量的函数

In [185]:
df1 = pd.DataFrame({'x':range(1,5),'y':range(11,15),'z':list('aabb')}, index=range(4))
grouper = df1.groupby(['z', list('mnmn')], axis=0)

In [186]:
"""单个函数，对应多列"""
grouper.agg(sum)

x   y
z         
a m  1  11
  n  2  12
b m  3  13
  n  4  14

In [187]:
"""多个函数，多应多列"""
grouper.agg([sum,max])

x       y    
    sum max sum max
z                  
a m   1   1  11  11
  n   2   2  12  12
b m   3   3  13  13
  n   4   4  14  14

In [188]:
"""为函数取名，以及可使用lambda、自定义函数"""
grouper.agg([('yoursum',sum),('mysum',lambda x:sum(x))])

x             y      
    yoursum mysum yoursum mysum
z                              
a m       1     1      11    11
  n       2     2      12    12
b m       3     3      13    13
  n       4     4      14    14

In [201]:
"""为每一列指定函数，不能为分组列指定函数"""
grouper.agg({'x':[('asum',sum),('amin',min)],'y':[('bsum',sum)]})

x         y
    asum amin bsum
z                 
a m    1    1   11
  n    2    2   12
b m    3    3   13
  n    4    4   14

### grouper之apply可接受更丰富的函数，比如返回多个值的函数

In [235]:
df1 = pd.DataFrame({'x':range(1,5),'y':range(11,15),'z':list('aabb')}, index=range(4))
grouper = df1.groupby(['z', list('mnmn')], axis=0)

In [236]:
def customf(x):
    return pd.Series([x.min(), x.max()], index=['min','max'])

In [237]:
"""只接受单个函数"""
grouper.apply(sum)  # 等价于grouper.agg(sum)

x   y  z
z            
a m  1  11  a
  n  2  12  a
b m  3  13  b
  n  4  14  b

In [238]:
grouper.apply(customf)  # grouper.agg(customf)会报错ValueError: Function does not reduce

min  \
z                                            
a m  x     1
y    11
z     a
dtype: object   
  n  x     2
y    12
z     a
dtype: object   
b m  x     3
y    13
z     b
dtype: object   
  n  x     4
y    14
z     b
dtype: object   

                                       max  
z                                           
a m  x     1
y    11
z     a
dtype: object  
  n  x     2
y    12
z     a
dtype: object  
b m  x     3
y    13
z     b
dtype: object  
  n  x     4
y    14
z     b
dtype: object

In [3]:
"""grouper.apply()使用lambda有歧义，还是grouper.egg()使用lambda靠谱，不过egg只适用聚合函数
groupber.apply()使用函数相对靠谱"""
def mx(x):
    return x.max()
df1 = pd.DataFrame({'x':list('aabb'), 'y':range(4), 'u':range(11,15),})
grouper = df1.groupby('x')

In [7]:
print(grouper.agg(lambda x: max(x)))  # return dataframe
print(grouper.apply(lambda x: max(x)))  # return series 有歧义，max(dataframe)得到的是最大的列名
print(grouper.agg(max))
print(grouper.apply(max))
print(grouper.agg(mx))
print(grouper.apply(mx))

   y   u
x       
a  1  12
b  3  14
x
a    y
b    y
dtype: object
   y   u
x       
a  1  12
b  3  14
   y   u
x       
a  1  12
b  3  14
   y   u
x       
a  1  12
b  3  14
   y   u
x       
a  1  12
b  3  14


### grouper之transform用于将结果与原df拼在一起，所以只接受标量函数，或结果与原df等长的函数，以便拼接。

In [239]:
df1 = pd.DataFrame({'x':range(1,5),'y':range(11,15),'z':list('aabb')}, index=range(4))
grouper = df1.groupby(['z'], axis=0)

In [240]:
"""只接受单个函数"""
pd.concat([df1, grouper.transform(sum)], axis=1)

,x,y,z,x,y
0,1,11,a,3,23
1,2,12,a,3,23
2,3,13,b,7,27
3,4,14,b,7,27


In [276]:
pd.concat([df1, grouper.transform(lambda x:x.iloc[[0,1]])], axis=1)

,x,y,z,x,y
0,1,11,a,1,11
1,2,12,a,2,12
2,3,13,b,3,13
3,4,14,b,4,14


### grouper的as_index与group_keys

In [283]:
df1 = pd.DataFrame({'x':range(1,5),'y':range(11,15),'z':list('aabb')}, index=range(4))
grouper = df1.groupby(['z'], axis=0)
"""group_keys=False 可以禁用分组键所形成的索引，不会删去原始对象的索引。
as_index=False 可以禁用分组键作为索引的行为，同时自动给定一个索引。
当两者都是False的情况下，相同之处在于：都会禁用分组键。不同之处在于：as_index在消除分组键的同时会自动生成一个索引。
当调用聚合函数时，其本身的索引会失效，此时传递group_keys=False无效(与group_keys=True一样)
"""
grouper_grpkey = df1.groupby(['z'], axis=0, group_keys=False)
grouper_index = df1.groupby(['z'], axis=0, as_index=False)

In [297]:
grouper.sum()

,x,y
z,,
a,3,23
b,7,27


In [298]:
grouper.agg(sum)

,x,y
z,,
a,3,23
b,7,27


In [299]:
grouper.apply(sum)

,x,y
z,,
a,3,23
b,7,27


In [300]:
"""对于聚合函数，group_keys=False无效"""
grouper_grpkey.sum()

,x,y
z,,
a,3,23
b,7,27


In [301]:
grouper_grpkey.agg(sum)

,x,y
z,,
a,3,23
b,7,27


In [302]:
grouper_grpkey.apply(sum)

,x,y
z,,
a,3,23
b,7,27


In [303]:
"""as_index=False 可以禁用分组键作为索引的行为，同时自动给定一个索引。"""
"""as_index=False还可以让用于分组的列保留在结果中"""
grouper_index.sum()

,z,x,y
0,a,3,23
1,b,7,27


In [304]:
grouper_index.agg(sum)

,z,x,y
0,a,3,23
1,b,7,27


In [305]:
"""apply中的sum可用于字符串相加"""
grouper_index.apply(sum)

,x,y,z
0,3,23,aa
1,7,27,bb


In [284]:
"""非聚合函数，原始结果"""
grouper.apply(lambda x:x.iloc[[0,1]])

x   y  z
z            
a 0  1  11  a
  1  2  12  a
b 2  3  13  b
  3  4  14  b

In [285]:
"""非聚合函数，group_keys=False有效，禁用分组键所形成的索引，不会删去原始对象的索引。"""
grouper_grpkey.apply(lambda x:x.iloc[[0,1]])

,x,y,z
0,1,11,a
1,2,12,a
2,3,13,b
3,4,14,b


In [286]:
"""非聚合函数，as_index=False 可以禁用分组键作为索引的行为，同时自动给定一个索引。"""
grouper_index.apply(lambda x:x.iloc[[0,1]])

x   y  z
0 0  1  11  a
  1  2  12  a
1 2  3  13  b
  3  4  14  b

# apply和map

In [4]:
df1 = pd.DataFrame(np.arange(6).reshape(2,3),columns=list('xyz'),index=list('ab'))
def customf(x):
    return pd.Series([x.min(), x.max()], index=['min','max'])
"""apply可接受自定义函数，lambda函数，内置函数。可指定axis。数据类型要能满足运算，否则报错"""
df1.apply(customf,axis=0)  # 每列计算返回series，所有列会组成dataframe返回

,x,y,z
min,0,1,2
max,3,4,5


In [56]:
df1.apply(lambda x:x.max() - x.min(), axis=0)

x    3
y    3
z    3
dtype: int64

In [57]:
"""applymap是应用到每个单元格"""
df1.applymap(lambda x: '%.1f' % x)

,x,y,z
a,0.0,1.0,2.0
b,3.0,4.0,5.0


In [58]:
"""Series可以map字典，但df的applymap不支持字典。map字典常用于修改df一列中的某些值"""
df1['y'].map(lambda x: x*2)  # create new
df1['y'].map({1:'a1',4:'a4'})  # create new

a    a1
b    a4
Name: y, dtype: object